In [1]:
import duckdb as ddb

In [2]:
ddb.sql("INSTALL httpfs; LOAD httpfs")

In [3]:
conn = ddb.connect('../airquality.db')

In [4]:
conn.execute("CREATE schema IF NOT EXISTS raw")

In [5]:
conn.sql("""
         SET s3_access_key_id='';
         SET s3_secret_access_key='';
         SET s3_region='';
         """)

In [6]:
conn.execute("""
             CREATE TABLE IF NOT EXISTS raw.air_quality_data(
                 location_id BIGINT,
                 sensor_id BIGINT,
                 "location" VARCHAR,
                 "datetime" TIMESTAMP,
                 LAT DOUBLE,
                 lon DOUBLE,
                 "parameter" VARCHAR,
                 units VARCHAR,
                 "value" DOUBLE,
                 "month" VARCHAR,
                 "year" BIGINT,
                 ingestion_datetime TIMESTAMP
             );
             """)

In [7]:
conn.execute("""
INSERT INTO raw.air_quality_data
SELECT 
    location_id, 
    sensors_id, 
    "location", 
    "datetime", 
    lat, 
    lon, 
    "parameter", 
    units, 
    "value",
    "month", 
    "year",
    current_timestamp AS ingestion_datetime
FROM read_csv('s3://openaq-data-archive/records/csv.gz/country=/locationid=222830/year=2021/month=04/*.csv.gz');
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [8]:
conn.close()